## Problem Statement/Goal
 - 

## Future Tasks:
- Scrape all the info from the website
 
## Completed Tasks:
- Scrape
    - List by just straight up adding the number to 6
    - Grab all from each list
    - add: ?part=single per page
- Extra info grabbing
    - Time in year and Quarter
    - Company Name in Ticker
    - Speaker Name in CEO or CFO
- Have the sections seperated between Main Speech and Q/A Sections
    - Sentiment
    - Complexity in Gunning Fog Score
 
## Conquered Challenges:
- Challenge:
    -Recieving a captcha
        - Changed the user agent
        - Made the user agent change everytime the browser was launched
        - Slowed the scraper's roll by 20 per round
    - Timeout Error
        - Put in a SINGLE Try/Except loop that had 5 to 20 counts of it. 
    - Solution(s):

## Imports

In [2]:
# Standard Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Scraping Libraries
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from time import sleep
import re

# NLP Libraries
from textblob import TextBlob
import textstat

## Functions

In [3]:
def Sentamentize(text):
    return TextBlob(str(text)).sentiment.polarity

def open_browser(alt_user_name = 'Thank you for your website'):
    opts = Options()
    opts.add_argument("user-agent=" + str(alt_user_name))
    path = '../Garage/chromedriver'         # Path to Chromedriver
    return webdriver.Chrome(executable_path = path, options=opts)

## Data Collection

In [ ]:
# Scraping urls of each transcript before the next step.
# Included were printouts of the page number the url was found on.
# I caught just over 6,000 urls at the time & stopped because of captchas
b_url='https://seekingalpha.com/earnings/earnings-call-transcripts/'
for page_num in range(1,1001):
    browser = open_browser("Y'all are great " + str(page_num))
        # I put in a "unique" user agent name to help throw
        # off the captcha.  This worked for only so long.
    current_ts_list = b_url + str(page_num)
    browser.get(current_ts_list)
    elements_list = browser.find_elements_by_class_name('dashboard-article-link')
    urls    = [el.get_attribute('href') + '?part=single' for el in elements_list]
    headers = [el.text for el in elements_list]
    print('scraping', page_num)
    #sleep(20)
    for transcript_num in range(len(urls)):
        current_transcript = {
            'header'   : headers[transcript_num],
            'url'      : urls[transcript_num],
            'page_list': page_num
        }
        url_list.append(current_transcript)
    browser.close()

In [ ]:
# Scraping every individual url in the df to return the full 
# transcript to be split and analyzed later.
for i in range(20):  # This allows for timeout errors w/oo crashing
    try:
        for row_num in range(len(url_df['transcript'])):
            if url_df['transcript'][row_num] == 0:
                
                # Go to url and obtain the full transcript
                browser = open_browser()
                browser.get(url_df['url'][row_num])
                print('scraping',row_num,
                    datetime.fromtimestamp(time()
                    ).strftime('%H:%M:%S'),
                    url_df['header'][row_num][:85]) # End of print
                
                soup    = BeautifulSoup(browser.page_source)
                article = soup.find('article')
                url_df['transcript'][row_num] = [item.text for 
                    item in article.find_all('p')]
                browser.close()
                sleep(20)
            else:
                pass
    except:
        print('--------')
        print('ERROR', i)
        print('--------')
        sleep(100)
        browser.close()
        #### Suggestions for the future scraper.
        # if browser:
        #    browser.close()
        # else:
        #    print("""Browser was already closed.  
        #    This wasn't a timeout error""")
        # For info on catching the errors that we want try:
        # https://stackoverflow.com/questions/33239308/how-to-
        # get-exception-message-in-python-properly/33239954

In [35]:
url_df = pd.read_csv('./data/url_df.csv')


## Viewing of Raw Data

The speech and QA columns in this were a mistake from a previous attempt.  They are written over later.

In [36]:
url_df.head(3)

,header,page_list,url,speech,QA,transcript
0,Vocera Communications (VCRA) CEO Brent Lang on...,NaN,https://seekingalpha.com/article/4239701-vocer...,NaN,"['Question-and-Answer Session', 'Operator', '[...",['Vocera Communications (NYSE:VCRA) Q4 2018 Ea...
1,Klepierre SA (KLPEF) CEO Jean-Michel Rene Gaul...,NaN,https://seekingalpha.com/article/4239700-klepi...,Jean-Michel Rene Gault - Deputy Chief Executiv...,"['Question-and-Answer Session', 'Q - Michel Va...",['Klepierre SA (OTCPK:KLPEF) Q4 2018 Earnings ...
2,ESCO Technologies Inc. (ESE) CEO Vic Richey on...,NaN,https://seekingalpha.com/article/4239698-esco-...,Vic Richey - Chairman and Chief Executive Offi...,"['Question-and-Answer Session', 'Operator', 'T...",['ESCO Technologies Inc. (NYSE:ESE) Q1 2019 Ea...


In [38]:
len(url_df['QA'][0])

28336

The original transcript columns were filled with lists rather than just strings.  This allowed me to seperate the speech and QA sections allong the string value "Question-and-Answer Session".  Unfortunatly, upon being saved these values have been transformed into simple strings.  Luckily, the splitting has already taken place beforehand.  But wont be super able to be shown for the future.   

The remainder of the code ahs been left in place to show how these things were done in the first place as best as possible.

## Cleaning

In [8]:
df = url_df.head(6000) # Rounding the number to 6,000.

In [9]:
df.drop(['page_list', 'url'], axis = 1, inplace = True)

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


I recieved a lot of these copy warnings during this project and I'll have to look into them more later.

In [14]:
# Creating new columns on the split at the QAS
# And then joining those chunks together.

# I originally had a for-loop that had to find the specific instance
# of the QAS but then discovered .index() and that made life easier.

QAS = 'Question-and-Answer Session'

# Defining a function to return the speech section.
def speech_function(x):
    if QAS in x:
        result = ''.join([words for words in x[1:x.index(QAS)]if len(words) >= 100])
    else:
        result = np.nan
    return result

# Defining a function to return the Q and Q section.
def QA_sec_function(x):
    if QAS in x:
        result = ''.join([words for words in x[x.index(QAS) +1:] if len(words) >= 25])
    else:
        result = np.nan
    return result

df['speech'] = df['transcript'].map(lambda x: speech_function(x))
df['QA']     = df['transcript'].map(lambda x: QA_sec_function(x))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## Performing Requested Analysis

In [778]:
# Performing the actual analysis that was requested 
# on the two different sections.
df['speech'].fillna('', inplace = True)
df['QA'].fillna('', inplace = True)
print("na's filled")

# Creating a sentiment columns from the speech & QA columns
df['speech_sentiment']  = df['speech'].map(lambda x: 
                                    Sentamentize(x))
print('speech_sentiment Done')
df['QA_sentiment']      = df['QA'].map(lambda x: 
                                    Sentamentize(x))
print('QA_sentiment Done')

# Creating a complexity column from the speech & QA columns
df['speech_complexity'] = df['speech'].map(lambda x: 
                                    textstat.gunning_fog(x))
print('speech_complexity Done')
df['QA_complexity']     = df['QA'].map(lambda x: 
                                    textstat.gunning_fog(x))
print('QA_complexity Done')


/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


na's filled


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


speech_sentiment Done


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


QA_sentiment Done


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


speech_complexity Done
QA_complexity Done


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [781]:
df.to_csv('./../EC_Analyzed.csv', index = False)

In [40]:
df = pd.read_csv('./data/EC_Analyzed.csv')

In [156]:
df.head(3)

,header,speech,QA,transcript,speech_sentiment,QA_sentiment,speech_complexity,QA_complexity,time,company,speaker
0,Vocera Communications (VCRA) CEO Brent Lang on...,"Good afternoon, ladies and gentlemen and welco...",[Operator Instructions] Your first question is...,['Vocera Communications (NYSE:VCRA) Q4 2018 Ea...,0.169205,0.110270,15.64,12.72,Q4 2018,Vocera Communications (VCRA),CEO Brent Lang
1,Klepierre SA (KLPEF) CEO Jean-Michel Rene Gaul...,"Good morning, everyone. Jean-Michel and myself...","Michel Varaldo, Société Générale. Two question...",['Klepierre SA (OTCPK:KLPEF) Q4 2018 Earnings ...,0.172358,0.139347,13.58,11.69,Q4 2018,Klepierre SA (KLPEF),CEO Jean-Michel Rene Gault
2,ESCO Technologies Inc. (ESE) CEO Vic Richey on...,Good day and welcome to the ESCO Technologies ...,Thank you. [Operator Instructions] And our fir...,['ESCO Technologies Inc. (NYSE:ESE) Q1 2019 Ea...,0.144630,0.188894,19.51,14.81,Q1 2019,ESCO Technologies Inc. (ESE),CEO Vic Richey


## Correcting 

In [151]:
def chunk(regex, x, speaker = False):
    string = re.search(regex, x)
    if string == None:
        string = np.nan
    else:
        string = string.group()
        
    if type(string) == str and speaker == True:
        string = string.strip(' on').strip('] ').strip(') ')
    return string

# Returns the time in quarter/year
df['time']    = df['header'].map(lambda x: chunk(r'Q\d \d+',x))
# Returns the Company Name
df['company'] = df['header'].map(lambda x: 
                        chunk(r'.+(\(|\[)[A-Z.:,-]+(\)|\])',x))
# Returns Speaker & Title
df['speaker'] = df['header'].map(lambda x: 
                        chunk(r'(\)|\]) .+ on', x, True))

Used the following code to identify which strings hadn't gotten picked up by my original regex finder(s).  I then went back and improved my  code as neccessary, however I was only able to clean and improve so much for the speaker column (especially when many weren't  listed) and the time column was awash in a virety of different  formats and  as such I decided not to bother myself with attempting to untangle  that web.

In [127]:
mask = df['company'].isna()
for i in df[mask]['header']:
    print(i)
    print('-----')

HollyFrontier Presents at Acquisition of Sonneborn M&A Conference - Transcript
-----


In [157]:
final_df = pd.DataFrame(df, columns = [
    'time',
    'company',
    'speaker',
    'speech_sentiment',
    'speech_complexity',
    'QA_sentiment',
    'QA_complexity',
    'header'
])
final_df.head()

,time,company,speaker,speech_sentiment,speech_complexity,QA_sentiment,QA_complexity,header
0,Q4 2018,Vocera Communications (VCRA),CEO Brent Lang,0.169205,15.64,0.110270,12.72,Vocera Communications (VCRA) CEO Brent Lang on...
1,Q4 2018,Klepierre SA (KLPEF),CEO Jean-Michel Rene Gault,0.172358,13.58,0.139347,11.69,Klepierre SA (KLPEF) CEO Jean-Michel Rene Gaul...
2,Q1 2019,ESCO Technologies Inc. (ESE),CEO Vic Richey,0.144630,19.51,0.188894,14.81,ESCO Technologies Inc. (ESE) CEO Vic Richey on...
3,Q1 2019,Electrovaya Inc. (EFLVF),Management,0.183982,18.14,0.000000,13.38,Electrovaya Inc. (EFLVF) Management on Q1 2019...
4,Q4 2018,Great-West Lifeco's (GWLIF),CEO Paul Mah,0.066531,18.80,0.105221,10.08,Great-West Lifeco's (GWLIF) CEO Paul Mahon on ...


In [158]:
final_df.to_csv('./data/Final_df.csv', index = False)

Breaking up the csv's so I can easily save them to github.

Maybe.  First I'm just going to put them elsewhere.

In [ ]:
pd.read_csv('./data/')